In [30]:
import pandas as pd
from pathlib import Path 

### CHANGE THESE ###
ground_truth_dir = "data/test"
prediction_dir = "output/transkribus_uten_base/"
model_name = "transkribus_uten_base"
### ------------ ###

prediction_dir = Path(prediction_dir)
ground_truth_dir = Path(ground_truth_dir)
metadata_df = pd.read_csv(prediction_dir / "metadata.csv")

# Det har skjedd noe rart med boksene...
gt_txt_files = {e for e in ground_truth_dir.iterdir() if e.name.endswith(".txt")}
predction_txt_files = {ground_truth_dir / f"{Path(e.word_image).stem}.gt.txt" for e in metadata_df.itertuples()}

assert len(gt_txt_files) == len(predction_txt_files)

pred_ = sorted(list(predction_txt_files - gt_txt_files))
gt_ = sorted(list(gt_txt_files - predction_txt_files))
pred_to_gt_map = {k: v for k, v in zip(pred_, gt_)}

transcriptions = {"model_name": [], "image": [], "transcription": [], "ground_truth": []}

for e in metadata_df.itertuples():
    line_img_stem = Path(e.word_image).stem
    gt_text_file = ground_truth_dir / f"{line_img_stem}.gt.txt"
    if not gt_text_file.exists():
        gt_text_file = pred_to_gt_map[gt_text_file]
    assert gt_text_file.exists()

    transcriptions["image"].append(e.word_image)
    transcriptions["model_name"].append(model_name)
    transcriptions["transcription"].append(e.word)
    transcriptions["ground_truth"].append(gt_text_file.read_text())

df = pd.DataFrame(transcriptions)
df.to_csv(f"output/{ground_truth_dir.name}_{model_name}_transcriptions_gt.csv", index=False)